# #11 Create a simply supported line beam model
<i>Run the whole code to generate and solve the model and display the minimum sagging moment, 3 different ways</i>
***

![Preview](../../_img/jupyter_notebook_051.png)

Define beam dimensions and loading:

In [ ]:
# Input Params
length = 10.0 #m		Length of beam
loading = 10.0  #kN/m   Uniformly distributed load

Connect to running Lusas instance:

In [ ]:
import sys; sys.path.append('../') # Reference modules in parent directory
from shared.LPI_21_1 import *
lusas = get_lusas_modeller()

# Check if unsaved model is open
if lusas.existsDatabase() and lusas.db().isModified():
    raise Exception("This script will create a new model. Please save or close the current model and try again")

Create new model:

In [ ]:
# Create a new model
lusas.newProject("Structural", "Simple_beam_model")
# Reference to the model database for convenience
db = lusas.database() 
# Set the vertical axis
db.setAnalysisCategory("3D")
db.setVerticalDir("Z")
# Set the unit system
db.setModelUnits("kN,m,t,s,C")

<div class="alert alert-block alert-info">
<b>Tip:</b> When interacting with Modeller externally, as we are here. A large speed up can be achived by disabling the user interface. In v22 and later this can be done with the enableUI function, in earlier versions the application must be set to be invisible. Take care to restore the visibility and watch out for exceptions in the code that can leave Modeller running in the background or disabled. See the code below to ensure this is properly restored.
</div>

In [ ]:
do_speed_up = False # To speed up interaction with Modeller from python set this to True

if do_speed_up: 
    if lusas.getMajorVersionNumber() >= 22:
        lusas.enableUI(False)
    else:
        lusas.setVisible(False)

Create the beam line:

In [ ]:
''' Create a line'''
# Get the geometry data object
geom_data = lusas.geometryData()
# Set the defaults
geom_data.setAllDefaults()
# Set the coordinates of the first point
geom_data.addCoords(0, 0, 0)
# Set the coordinates of the second point
geom_data.addCoords(length, 0, 0)
# Create the line object from the geometry data
objs = db.createLine(geom_data) # Modeller returns an object set containing all the lines created, in this case we only have 1.
# Get the lines
lines = objs.getObjects("Lines")
# Get the single line from the returned array
beam_line :IFLine = lines[0]

<div class="alert alert-block alert-info">
<b>Tip:</b> For an easier way to create geometries using the Helpers.py see example #03.
</div>

Create attributes:
- an 1m line mesh attibute,
- a geometric attibute with custom section properties,
- a steel material attribute,
- two support attributes (fixed and pinned),
- a uniform distributed load (UDL) attribute

In [ ]:
''' Create a mesh attribute'''
mesh_attr = db.createMeshLine("Beam Mesh")
# Set the element type and number of elements (3 thick beam elements here)
mesh_attr.setNumber("BMI21", 3)

In [ ]:
''' Create a geometric attribute'''
geometric_attr = db.createGeometricLine("Beam Geometry")
# Set the beam section properties
geometric_attr.setBeam(0.0125, 0.0004573, 0.00002347, 0.0, 0.00000121, 0.00532608, 0.00755776, 0.0, 0.0, 0)

In [ ]:
''' Create a material attribute'''
material_attr = db.createIsotropicMaterial("Steel", 209_000_000, 0.3, 7.8)

In [ ]:
''' Create a support attribute (fixed) '''
fixed_support_attr = db.createSupportStructural("Fixed")
# set the freedoms F=Free, R=Restrained
fixed_support_attr.setStructural("R", "R", "R", "F", "F", "F", "F", "F", "F")

''' Create a support attribute (pinned) '''
pinned_support_attr = db.createSupportStructural("Pinned")
# set the freedoms 
pinned_support_attr.setStructural("F", "R", "R", "F", "F", "F", "F", "F", "F")

In [ ]:
''' Create a beam load attribute '''
udl_load_attr = db.createLoadingBeamDistributed("UDL")
udl_load_attr.setBeamDistributed("Parametric", "Global", "beam")
udl_load_attr.addRow(0.0, 0.0, 0.0, -loading, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, -loading, 0.0, 0.0, 0.0)

Now assign all the created attributes on the beam line:

In [ ]:
''' Assign the attributes to the geometry '''
# get the assignment object
assignment = lusas.assignment()
# set the defaults
assignment.setAllDefaults()

# Assign the mesh
mesh_attr.assignTo(beam_line, assignment)
# Assign the geometry
geometric_attr.assignTo(beam_line, assignment)
# Assign the material
material_attr.assignTo(beam_line, assignment)
# Assign the loading
udl_load_attr.assignTo(beam_line, assignment)

''' Assign the supports to the points of the line '''
# Assign the fixed support to the first point
fixed_support_attr.assignTo(beam_line.getStartPoint(), assignment)
# Assign the pinned support to the last point
pinned_support_attr.assignTo(beam_line.getEndPoint(), assignment)

<H2>Solving the Analysis</H2>

In [ ]:
# Save the model
db.save()

# Solve all defined analyses
allAnalyses : list[IFAnalysisBaseClass] = db.getAnalyses()
for analysis in allAnalyses:
    analysis.solve(True)

db.openAllResults(True, True)

Find max bending:

In [ ]:
''' Successful analysis - Process the results to determine the max bending '''
import sys
min_moment = sys.float_info.max
nodeNum = 0
# Get the results at each node to determine the max
allElements : list[IFElement] = beam_line.getElements()
for element in allElements:
    for node in element.getNodes():
        # Extract the nodal result for the required Entity and Component
        my = node.getResults("Force/Moment - Thick 3D Beam", "My")[0] # returns a tuple, take the actual value
        # Save the minimum (sagging) moment -ve in LUSAS
        if my < min_moment :
            min_moment = my
            nodeNum = node.getID()

In [ ]:
# Get the units of the current model for display
forceUnit = db.getModelUnits().getForceShortName()
lengthUnit = db.getModelUnits().getLengthShortName()

# Print the result
print("Max Sagging Bending Moment (Nodal) = {0:.1f}{1}{2} at node {3}".format(min_moment, forceUnit, lengthUnit, nodeNum) )

In [ ]:
# BMI21 Elements have results at "internal points" between nodes, get the results here too
min_moment2 = sys.float_info.max
elementNum = 0
internalPnt = 0
for element in allElements:
	if element.shouldUseInternalPoints(): # We know they should but good practice to check in case this code is used elsewhere
		for i in range(0, element.countInternalPoints()):
			# Extract the nodal result for the required Entity and Component
			my = element.getInternalResults(i, "Force/Moment - Thick 3D Beam", "My")[0] # returns a tuple, take the actual value
			# Save the minimum (sagging) moment -ve in LUSAS
			if my < min_moment2 :
				min_moment2 = my
				elementNum = element.getID()
				internalPnt = i
				
# Print the result using the lusas string format
print("Max Sagging Bending Moment (Internal Point) = {0:.1f}{1}{2} at element {3}, ip {4}".format(min_moment2, forceUnit, lengthUnit, elementNum, internalPnt) )

Alternatively we can simply ask the line to do this work for us

In [ ]:
min_moment3 = beam_line.getResults("featureMin", "Force/Moment - Thick 3D Beam", "My", None, elementNum, internalPnt)

print("Max Sagging Bending Moment (Feature) = {0:.1f}{1}{2} at element {3}, ip {4}".format(min_moment2, forceUnit, lengthUnit, elementNum, internalPnt) )


<div class="alert alert-block alert-warning">
<b>Note:</b> If the code fails before this point the cell below must be run manually to restore the user interface
</div>

In [ ]:
if lusas.getMajorVersionNumber() >= 22:
    lusas.enableUI(True)
else:
    lusas.setVisible(True)